In [1]:
import dabench as dab
import numpy as np
import jax
from timeit import default_timer as timer
import pandas as pd

from ray import train, tune
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch

In [2]:
%%bash
mkdir -p out/l96

# Define parameters

In [7]:
system_dim= 36
spinup_size = 14400
valid_size = 5000
test_size = 5000
nr_steps = spinup_size + valid_size + test_size
delta_t=0.01
obs_sd = 0.5
sigma_bg = 0.3
sigma_obs = 0.625
analysis_window = 0.1
analysis_time_in_window = 0.05
obs_location_count = 18
random_seed = 5
num_iters = 3
n_outer_loops = 3 

# Function definition: Backprop 4DVar

We'll need to prep and run Backprop-4DVar many times, so this wraps it all into one function

In [8]:
def run_backprop_4dvar(system_dim, nr_steps, spinup_size, valid_size, test_size, 
                       test_run, delta_t, obs_location_count, obs_sd, sigma_bg, 
                       sigma_obs, analysis_window, analysis_time_in_window, 
                       random_seed, num_iters, learning_rate, lr_decay):
    np_rng = np.random.default_rng(random_seed)
    jax.clear_caches()

    ### Nature Run
    nature_run = dab.data.Lorenz96(system_dim=system_dim, delta_t=delta_t,
                                   store_as_jax=True, random_seed=random_seed)

    x0_initial = np_rng.normal(size=system_dim, scale=1)
    nature_run.generate(n_steps=nr_steps, x0 = x0_initial) 
    nr_spinup, nr_valid, nr_test = nature_run.split_train_valid_test(
        spinup_size, valid_size, test_size)

    if not test_run:
        nr_eval = nr_valid
    else:
        nr_eval = nr_test


    ### Observations
    obs_l96 = dab.observer.Observer(
        nr_eval,
        time_indices = np.arange(0, nr_eval.time_dim, 5),
        random_location_count = obs_location_count,
        error_bias = 0.0,
        error_sd = obs_sd,
        random_seed=random_seed,
        stationary_observers=True,
        store_as_jax=True
    )
    obs_vec_l96 = obs_l96.observe()

    
    ### Forecast Model
    model_l96 = dab.data.Lorenz96(system_dim=system_dim, delta_t=delta_t, 
                                  store_as_jax=True, random_seed=random_seed)

    class L96Model(dab.model.Model):                                                                       
        """Defines model wrapper for Lorenz96 to test forecasting."""
        def forecast(self, state_vec, n_steps):
            self.model_obj.generate(x0=state_vec.values, n_steps=n_steps)
            new_vals = self.model_obj.values 

            new_vec = dab.vector.StateVector(values=new_vals, store_as_jax=True)

            return new_vec

    fc_model = L96Model(model_obj=model_l96)
    
    ### Set up DA matrices: H (observation), R (obs error), B (background error)
    H = np.zeros((obs_location_count, system_dim))
    H[np.arange(H.shape[0]), obs_vec_l96.location_indices[0]] = 1
    R = (sigma_obs**2)* np.identity(obs_location_count)
    B = (sigma_bg**2)*np.identity(system_dim)

    
    ### Run data assimilation
    da_time_start = timer()
    
    # Prep DA object
    dc = dab.dacycler.Var4DBackprop(
        system_dim=system_dim,
        delta_t=nr_eval.delta_t,
        H=H,
        B=B,
        R=R,
        num_iters=num_iters,
        loss_growth_limit=5,
        learning_rate=learning_rate,
        lr_decay=lr_decay,
        model_obj=fc_model,
        obs_window_indices=[0,5,10],
        steps_per_window=11, # 11 instead of 10 because inclusive of 0 and 11
        )

    # Generate initial conditions
    cur_tstep = 0
    x0_original = nr_eval.values[cur_tstep] + np_rng.normal(size=(system_dim,), 
                                                            scale=1)
    x0_sv = dab.vector.StateVector(
        values=x0_original,
        store_as_jax=True)
    
    # Execute
    out_statevec = dc.cycle(
        input_state = x0_sv,
        start_time = nr_eval.times[cur_tstep],
        obs_vector = obs_vec_l96,
        analysis_window=analysis_window,
        timesteps=498,
        obs_error_sd=sigma_obs,
        analysis_time_in_window=analysis_time_in_window)
    
    da_time = timer()-da_time_start
    rmse = np.sqrt(np.mean(np.square(nr_eval.values[:-20] - out_statevec.values)))
    
    return out_statevec, rmse, obs_vec_l96, nr_eval, da_time

# Hyperparameter optimization on validation set

Using RayTune to optimize alpha (learning rate) and alpha decay

In [9]:
def train_backprop_4dvar(config, system_dim, num_obs, obs_sd):
    """A wrapper for RayTune to run backprop 4Dvar"""
    random_seed = system_dim 
    obs_location_count = num_obs
    run_dict = dict(
        system_dim=system_dim, 
        nr_steps=nr_steps,
        spinup_size=spinup_size,
        valid_size=valid_size,
        test_size=test_size,
        test_run=False,
        delta_t=delta_t,
        obs_location_count=obs_location_count,
        obs_sd=obs_sd,
        sigma_bg=obs_sd/1.5,
        sigma_obs=obs_sd*1.25,
        analysis_window=analysis_window,
        analysis_time_in_window=analysis_time_in_window,
        random_seed=random_seed,
        num_iters=num_iters,
        learning_rate=0,
        lr_decay = 0)

    run_dict['learning_rate'] = config['lr']
    run_dict['lr_decay'] = config['lr_decay']
    
    out_bp, error_bp, obs_vec_l96, nr_eval, da_time = run_backprop_4dvar(**run_dict)
    
    train.report({'rmse':error_bp})

### System size experiments

In [6]:
# Define search space
space = {
    "lr": hp.loguniform("lr", -10, 0),
    "lr_decay": hp.uniform("lr_decay", 0.1, 0.99),
}

In [7]:
all_results_df_list = []
system_dim_list = [6, 20, 36, 72, 144, 256]

for system_dim in system_dim_list:
    tune_time_start = timer()
    
    trainable_with_system_dim = tune.with_parameters(train_backprop_4dvar, 
                                                     system_dim=system_dim,
                                                     num_obs=int(system_dim/2),
                                                     obs_sd=0.5
                                                     )
    
    # Not running in parallel to get fairer time comparison against regular 4DVar
    trainable_with_resources = tune.with_resources(trainable_with_system_dim, {"cpu": 16})
    hyperopt_search = HyperOptSearch(space, metric="rmse", mode="min")
    tuner = tune.Tuner(
        trainable_with_resources,
        tune_config=tune.TuneConfig(
            num_samples=50,
            search_alg=hyperopt_search,
        ),
    )
    
    results = tuner.fit()

    cur_results_df = results.get_dataframe()
    cur_results_df['system_dim'] = system_dim
    tune_time = timer() - tune_time_start
    cur_results_df['total_tune_time'] = tune_time

    all_results_df_list.append(cur_results_df)

(train_backprop_4dvar pid=172253) /tmp/ipykernel_169542/1837075594.py:6: DeprecationWarning: jax.clear_backends is deprecated.


(train_backprop_4dvar pid=172253) [[ 28.07867463  28.07412639  28.07088946]
(train_backprop_4dvar pid=172253)  [ 71.57125221  71.550212    71.53523923]
(train_backprop_4dvar pid=172253)  [151.74321096 151.70264896 151.67378378]
(train_backprop_4dvar pid=172253)  ...
(train_backprop_4dvar pid=172253)  [273.30950889 273.26930963 273.24069862]
(train_backprop_4dvar pid=172253)  [208.25943168 208.22513458 208.20072482]
(train_backprop_4dvar pid=172253)  [183.21065004 183.17359444 183.14722231]]


(train_backprop_4dvar pid=174140) /tmp/ipykernel_169542/1837075594.py:6: DeprecationWarning: jax.clear_backends is deprecated.


(train_backprop_4dvar pid=174140)  [150.13942426 149.16726516 149.00121814]
(train_backprop_4dvar pid=174140)  [482.5793027  480.79438437 480.48859941]
(train_backprop_4dvar pid=174140)  [479.4559678  477.39235305 477.03900635]
(train_backprop_4dvar pid=174140)  [677.93424813 675.25120966 674.79169314]]
(train_backprop_4dvar pid=174140) [[ 28.07867463  27.96845592  27.94959719]
(train_backprop_4dvar pid=174140)  [ 71.27030986  70.76291536  70.67626381]
(train_backprop_4dvar pid=174140)  ...


(train_backprop_4dvar pid=174604) /tmp/ipykernel_169542/1837075594.py:6: DeprecationWarning: jax.clear_backends is deprecated.


...

(train_backprop_4dvar pid=176211) [[ 28.07867463  26.90213716  26.19806853]
(train_backprop_4dvar pid=176211)  [ 65.70196775  60.75008426  57.86266261]
(train_backprop_4dvar pid=176211)  [122.2329138  113.82162631 108.90820591]
(train_backprop_4dvar pid=176211)  ...
(train_backprop_4dvar pid=176211)  [310.96593196 297.63808016 289.55845101]
(train_backprop_4dvar pid=176211)  [223.31724365 214.82559955 209.6551506 ]
(train_backprop_4dvar pid=176211)  [119.57849178 114.18900252 110.93657296]]


(train_backprop_4dvar pid=176521) /tmp/ipykernel_169542/1837075594.py:6: DeprecationWarning: jax.clear_backends is deprecated.


(train_backprop_4dvar pid=176521) [[ 28.07867463  61.54478706  25.47938382]
(train_backprop_4dvar pid=176521)  [  2.7142332    3.7506663    2.39657495]] [repeated 5x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(train_backprop_4dvar pid=176521)  ...


(train_backprop_4dvar pid=176836) /tmp/ipykernel_169542/1837075594.py:6: DeprecationWarning: jax.clear_backends is deprecated.
2024-06-11 14:55:43,547	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-06-11 14:55:43,578	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/ksolvik/ray_results/train_backprop_4dvar_2024-06-11_14-49-25' in 0.0272s.


KeyboardInterrupt: 

In [ ]:
full_results_df = pd.concat(all_results_df_list)
full_results_df.to_csv('./out/l96/raytune_l96_hessian_v4.csv')

### Experiments varying number of observations and obs error

In [10]:
space = {
    "lr": hp.loguniform("lr", -5, 0),
    "lr_decay": hp.uniform("lr_decay", 0.1, 0.99),
}

In [ ]:
all_results_df_list = []

system_dim = 36
num_obs_list = [6, 12, 18, 24, 30, 36]
obs_error_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1.0, 1.5, 2.0]

for obs_location_count in num_obs_list:
    for obs_sd in obs_error_list:
        tune_time_start = timer()
        
        trainable_with_system_dim = tune.with_parameters(train_backprop_4dvar, 
                                                         system_dim=system_dim,
                                                         num_obs=obs_location_count,
                                                         obs_sd=obs_sd
                                                         )
        trainable_with_resources = tune.with_resources(trainable_with_system_dim, {"cpu": 4})

        hyperopt_search = HyperOptSearch(space, metric="rmse", mode="min")
        tuner = tune.Tuner(
            trainable_with_resources,
            tune_config=tune.TuneConfig(
                num_samples=20,
                search_alg=hyperopt_search,
            ),
        )
        
        results = tuner.fit()

        cur_results_df = results.get_dataframe()
        cur_results_df['system_dim'] = system_dim
        cur_results_df['num_obs'] = obs_location_count
        cur_results_df['obs_sd'] = obs_sd
        tune_time = timer() - tune_time_start
        cur_results_df['total_tune_time'] = tune_time
        
        all_results_df_list.append(cur_results_df)


In [ ]:
full_results_df = pd.concat(all_results_df_list)
full_results_df.to_csv('./out/l96/raytune_werrors_heatmap_hessian_v4_alldims_bigger_error.csv')